In [1]:
import pandas as pd

In [2]:
movies = pd.read_json("movies.json")
reviews = pd.read_json("reviews.json")

In [3]:
movies.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [4]:
reviews.head()

,user_id,item_id,rating,timestamp
0,0,50,5,1997-12-04 15:55:49
1,0,172,5,1997-12-04 15:55:49
2,0,133,1,1997-12-04 15:55:49
3,196,242,3,1997-12-04 15:55:49
4,186,302,3,1998-04-04 19:22:22


In [5]:
reviews = reviews.drop(['timestamp'], axis=1)

In [6]:
# Wrangle plan
# 1. Get the N movies that have the most reviews (reduces dimensionality and that's important when thinking about euclidean distance)
# 2. Use these top movies to cluster users based on their rating similarities
# 3. Make a recommendation based on what other users in the cluster have reviewed highly

In [ ]:
# get counts of each unique movie
# Note this will take about 30 minutes
counts = []
for movie in range(1682):
    tally = 0
    for idx in range(len(reviews)):
        if reviews.iloc[idx]['item_id'] == movie:
            tally = tally + 1
    counts.append(tally)
    print("Movie " + str(movie) + " done!")

In [8]:
#Let's do this instead
pivotDF = reviews.pivot(index='user_id', columns='item_id', values='rating')
pivotDF.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Get the item_id with count
mostReviewed = 3
topN = reviews.groupby('item_id').size()

In [29]:
topN.head()

item_id
1    450
2    131
3     90
4    208
5     86
dtype: int64

In [ ]:
# Get a dataframe with only topN movies included
topOnlyDF = reviews[reviews["item_id"].isin(topN["item_id"])]

In [ ]:
topOnlyDF.head() # Need that pivot again

In [ ]:
topOnlyDF = topOnlyDF.pivot(index='user_id', columns='item_id', values='rating')

In [ ]:
# People who don't review anything aren't useful to us. Let's drop them.
topOnlyDF = topOnlyDF.dropna(axis=0, how='all')

In [ ]:
# The people who are left might still have NaNs. Let's fill those with a number so we can cluster!
topOnlyDF = topOnlyDF.fillna(-1)

In [ ]:
topOnlyDF.head()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
agg = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
agg = agg.fit(topOnlyDF)
agg.labels_

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
# We want something as close to 1 as possible
ss_avg = silhouette_score(topOnlyDF, agg.labels_)

In [ ]:
ss_avg # Can we do better?

In [ ]:
scores = {}
clusters = len(topOnlyDF)
for i in range(2, clusters):
    agg = AgglomerativeClustering(n_clusters=i, affinity='euclidean', linkage='ward')
    agg = agg.fit(topOnlyDF)
    ss_avg = silhouette_score(topOnlyDF, agg.labels_)
    scores[i] = ss_avg

In [ ]:
import seaborn as sns
sns.scatterplot(x=scores.keys(), y=scores.values()) # Somewhere around 180

In [ ]:
# choose a n_cluster value
agg = AgglomerativeClustering(n_clusters=180, affinity='euclidean', linkage='ward')
agg = agg.fit(topOnlyDF)

In [ ]:
topOnlyDF["clusters"] = agg.labels_
topOnlyDF.reset_index(inplace=True)
topOnlyDF.head()

In [ ]:
# Let's look at a specific user right now
user = 365
topOnlyDF[topOnlyDF['user_id'] == user]

In [ ]:
# Now, let's get users in same cluster
cluster = 77
user_ids = topOnlyDF[topOnlyDF['clusters'] == cluster]
user_ids

In [ ]:
reviews[reviews["user_id"] == 24]

In [ ]:
u365 = reviews[reviews["user_id"] == 365]

In [ ]:
u365[u365['item_id'] == 64]

In [ ]:
movies[movies["item_id"] == 64]